In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")

In [ ]:
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

In [ ]:
inWidth=368
inHeight=368
thr=0.2

In [ ]:
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
from ipywidgets import interact, Button, Layout, HBox, FileUpload, VBox
import os

# Assuming BODY_PARTS, POSE_PAIRS, inWidth, inHeight, thr, and graph_opt.pb are defined elsewhere

# Function to calculate pose similarity
def calculate_pose_similarity(points1, points2):
    # Implement your pose similarity calculation logic here
    # This can be based on Euclidean distance between corresponding body parts, angles, etc.
    # Return a similarity score between 0 (different) and 1 (similar)

    # Example: Euclidean distance between corresponding body parts
    distances = [np.linalg.norm(np.array(p1) - np.array(p2)) if (p1 is not None and p2 is not None) else 0
                 for p1, p2 in zip(points1, points2)]

    # Normalize distances and calculate similarity score
    max_distance = np.sqrt(2 * (800**2))  # Max possible distance in an 800x800 frame
    similarity_score = 1 - (sum(distances) / (len(distances) * max_distance))

    return similarity_score

# Load the pose estimation model from graph_opt.pb
net = cv2.dnn.readNetFromTensorflow('graph_opt.pb')

# Define the folder path to store the pose images
poses_folder = 'C:\\Users\\hp\\Poses'

# Get the list of existing pose images
existing_poses = [f for f in os.listdir(poses_folder) if f.endswith('.jpg')]

# Create buttons and set an initial value for selected_pose_path
button_layout = Layout(width='50px', height='50px')
buttons = []
button_images = []

def start_pose_comparison(b):
    global selected_pose_path
    selected_pose_path = os.path.join(poses_folder, existing_poses[buttons.index(b)])
    print(f"Selected Pose Path: {selected_pose_path}")

# Create buttons and display them
for pose in existing_poses:
    img_path = os.path.join(poses_folder, pose)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (50, 50))
    img_widget = widgets.Image(value=cv2.imencode('.png', img)[1].tobytes(), format='png', width=50, height=50)
    button_images.append(img_widget)
    button = Button(description='', layout=button_layout, image=img_widget)
    button.on_click(start_pose_comparison)
    buttons.append(button)

# Set initial value for selected_pose_path
selected_pose_path = os.path.join(poses_folder, existing_poses[0])

# Create a FileUpload widget for uploading new poses
upload_pose_widget = FileUpload(accept='image/*', multiple=True)

# Function to handle the file upload
def handle_upload(change):
    global selected_pose_path
    uploaded_files = upload_pose_widget.value
    for filename, file_info in uploaded_files.items():
        img_path = os.path.join(poses_folder, filename)
        with open(img_path, 'wb') as f:
            f.write(file_info['content'])
        # Update the selected_pose_path with the newly uploaded photo
        selected_pose_path = img_path
        # Add the new pose to the list of existing poses and update the buttons
        existing_poses.append(filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (50, 50))
        img_widget = widgets.Image(value=cv2.imencode('.png', img)[1].tobytes(), format='png', width=50, height=50)
        button = Button(description='', layout=button_layout, image=img_widget)
        button.on_click(start_pose_comparison)
        buttons.append(button)
        button_images.append(img_widget)
        print(f"Uploaded Pose: {img_path}")

# Attach the handle_upload function to the file upload widget
upload_pose_widget.observe(handle_upload, names='value')

# Display buttons and upload widget
button_row = HBox(buttons)
image_row = HBox(button_images)
display(button_row, image_row, upload_pose_widget)


In [ ]:
# Assuming the selected_pose_path variable is already defined in the first cell

# Capture video from the live camera
cap = cv2.VideoCapture(0)
cap.set(3, 800)
cap.set(4, 800)

warning_messages = {
    BODY_PARTS['LWrist']: "Adjust your left wrist and arm",
    BODY_PARTS['RWrist']: "Adjust your right wrist and arm",
    BODY_PARTS['LAnkle']: "Adjust your left leg and ankle",
    BODY_PARTS['RAnkle']: "Adjust your right leg and ankle",
    BODY_PARTS['Neck']: "Adjust your head and neck",
}

# Main loop for live video comparison
while cv2.waitKey(1) < 0:
    has_frame, frame = cap.read()
    frame_width = frame.shape[1]
    frame_height = frame.shape[0]

    # Resize the live frame to match the height of the selected pose image
    frame_resized = cv2.resize(frame, (400, 800))

    # Perform pose estimation on the live frame
    net.setInput(cv2.dnn.blobFromImage(frame_resized, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out_live = net.forward()
    out_live = out_live[:, :19, :, :]

    points_live = []
    for i in range(len(BODY_PARTS)):
        heatMap = out_live[0, i, :, :]
        _, conf, _, point = cv2.minMaxLoc(heatMap)
        x = (400 * point[0]) / out_live.shape[3]
        y = (800 * point[1]) / out_live.shape[2]
        points_live.append((int(x), int(y)) if conf > thr else None)

    # Perform pose estimation on the selected pose image
    pose_img = cv2.imread(selected_pose_path)
    pose_img = cv2.resize(pose_img, (800, 800))

    net.setInput(cv2.dnn.blobFromImage(pose_img, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out_pose = net.forward()
    out_pose = out_pose[:, :19, :, :]

    assert(len(BODY_PARTS) == out_pose.shape[1])

    points_pose = []
    for i in range(len(BODY_PARTS)):
        heatMap = out_pose[0, i, :, :]
        _, conf, _, point = cv2.minMaxLoc(heatMap)
        x = (800 * point[0]) / out_pose.shape[3]
        y = (800 * point[1]) / out_pose.shape[2]
        points_pose.append((int(x), int(y)) if conf > thr else None)

    # Compare poses and calculate similarity score
    similarity_score = calculate_pose_similarity(points_pose, points_live)

    # Draw the expected pose on the live frame
    frame_resized_with_pose = pose_img.copy()

    for pair in POSE_PAIRS:
        part_from = pair[0]
        part_to = pair[1]
        assert(part_from in BODY_PARTS)
        assert(part_to in BODY_PARTS)

        id_from = BODY_PARTS[part_from]
        id_to = BODY_PARTS[part_to]

        if points_pose[id_from] and points_pose[id_to]:
            cv2.line(frame_resized_with_pose, points_pose[id_from], points_pose[id_to], (0, 255, 0), 3)
            cv2.ellipse(frame_resized_with_pose, points_pose[id_from], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
            cv2.ellipse(frame_resized_with_pose, points_pose[id_to], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)

    # Draw the live pose on the live frame
    for pair in POSE_PAIRS:
        part_from = pair[0]
        part_to = pair[1]
        assert(part_from in BODY_PARTS)
        assert(part_to in BODY_PARTS)

        id_from = BODY_PARTS[part_from]
        id_to = BODY_PARTS[part_to]

        if points_live[id_from] and points_live[id_to]:
            cv2.line(frame_resized, points_live[id_from], points_live[id_to], (0, 255, 0), 3)
            cv2.ellipse(frame_resized, points_live[id_from], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
            cv2.ellipse(frame_resized, points_live[id_to], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)

    # Display the live frame with the drawn expected and live poses
    combined_frame = np.concatenate((frame_resized, frame_resized_with_pose), axis=1)

    # Display instructions on the live frame
    if similarity_score > 0.95:  # Adjust the threshold as needed
        cv2.putText(combined_frame, "Your pose is good!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        # Display warnings for detected issues
        for part_id, warning_message in warning_messages.items():
            if points_live[part_id] is None:
                cv2.putText(combined_frame, warning_message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                break  # Display only one warning at a time

    # Display the live frame in another window
    cv2.imshow('Live Pose Comparison', combined_frame)

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()
